In [1]:
import torch
import pandas as pd
import os
import sqlite3
import matplotlib.pyplot as plt
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from sklearn.preprocessing import OneHotEncoder

In [ ]:
# frame_id: 0
# event_subtype_id: 0
# frame_start:0
# frame_end: 0
# player_id: 0
# x_start: 0
# y_start: 0
# x_end: 0
# y_end: 0
# // team_id: 0
# player_in_possession_id: 0
# player_possession_frame_start: 0
# player_in_possession_x_start: 0
# player_in_possession_y_start: 0
# player_in_possession_x_end: 0
# player_in_possession_y_end: 0

In [2]:
MATCH_ID = "978805"
EVENT_DATA_DIR = "myairbridge-6YJaD9uGRDr/machine learning data/events"
MATCH_ID_LIST = [ i.split("_")[0] for i in os.listdir(EVENT_DATA_DIR) ]
TRACKING_DATA_DIR = "myairbridge-6YJaD9uGRDr/machine learning data/tracking"
N_EPOCHS = 10

In [16]:
def load_events(m_id:str) -> pd.DataFrame:
    events_raw = pd.read_csv(os.path.join(EVENT_DATA_DIR, f"{m_id}_events.csv"), index_col=0,)
    events = events_raw# [["frame_start", "frame_end", "event_subtype"]]

    return events

In [7]:
def load_tracking(m_id:str) -> pd.DataFrame:
    tracking_raw = pd.read_csv(os.path.join(TRACKING_DATA_DIR, f"{m_id}_tracking.csv"))
    tracking = tracking_raw.drop(columns=["match_id", "half", "timestamp"])
    tracking_ball = tracking.loc[tracking.object_id == -1].drop(columns=["object_id", "extrapolated"])
    return tracking_ball

In [17]:
events = load_events(MATCH_ID)
tracking_ball = load_tracking(MATCH_ID)

In [18]:
events

,match_id,frame_start,frame_end,time_start,time_end,period,attacking_side_id,attacking_side,minute_start,second_start,...,push_defensive_line,inside_defensive_shape_start,inside_defensive_shape_end,separation_start,separation_end,separation_gain,received_in_space,opponents_ahead_start,opponents_ahead_end,opponents_overtaken
index,,,,,,,,,,,,,,,,,,,,,
0,978805.0,320,350,00:22.0,00:25.0,1,2,right_to_left,0,22,...,False,True,False,3.96,2.45,-1.51,False,6,8,-2
1,978805.0,356,370,00:25.6,00:27.0,1,2,right_to_left,0,25,...,False,True,True,3.29,4.52,1.24,False,2,4,-2
2,978805.0,360,377,00:26.0,00:27.7,1,2,right_to_left,0,26,...,False,True,False,4.51,2.00,-2.51,False,4,1,3
3,978805.0,440,451,00:34.0,00:35.1,1,1,left_to_right,0,34,...,False,True,True,3.02,4.27,1.24,True,7,6,1
4,978805.0,446,466,00:34.6,00:36.6,1,1,left_to_right,0,34,...,False,True,True,0.68,2.38,1.70,False,6,4,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
419,978805.0,56497,56567,92:09.7,92:16.7,2,1,left_to_right,92,9,...,False,False,False,7.06,3.18,-3.88,False,5,3,2
420,978805.0,56501,56567,92:10.1,92:16.7,2,1,left_to_right,92,10,...,False,True,True,2.62,1.82,-0.81,False,7,5,2
421,978805.0,57445,57460,93:44.5,93:46.0,2,2,right_to_left,93,44,...,False,True,True,3.77,3.41,-0.37,False,6,5,1


In [ ]:
events.columns

In [11]:
def preprocess(events:pd.DataFrame, tracking:pd.DataFrame):
    conn = sqlite3.connect(':memory:')
    #write the tables
    events.to_sql('events', conn, index=False)
    tracking_ball.to_sql('tracking_ball', conn, index=False)

    qry = '''
        select 
            *
            -- x,y,z, frame_id, event_subtype
        from
            events right join tracking_ball on
            tracking_ball.frame_id between frame_start and frame_end
        '''
    
    df = pd.read_sql_query(qry, conn)

    return df
df = preprocess(events, tracking_ball)

In [14]:
X = df[['x', 'y', 'z']]
y = df['event_subtype']

In [ ]:
(y.value_counts()/len(y)).plot(kind='bar')

In [21]:
X_tensor = torch.tensor(X.values, dtype=torch.float32).unsqueeze(0)

In [20]:
classes = y.unique()
n_classes = len(classes)

In [21]:
n_classes

11

In [22]:
ohe = OneHotEncoder()
y_ohe = torch.tensor(ohe.fit_transform(y.values.reshape(-1, 1)).toarray()).unsqueeze(0)

In [23]:
y_ohe

tensor([[[0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         ...,
         [0., 0., 0.,  ..., 0., 0., 1.],
         [0., 0., 0.,  ..., 0., 0., 1.],
         [0., 0., 0.,  ..., 0., 0., 1.]]], dtype=torch.float64)

In [24]:
class BiLSTM(nn.Module):

    def __init__(self, input_dim:int, hidden_dim:int, batch_size:int, output_dim:int=11, num_layers:int=2):
        super(BiLSTM, self).__init__()
        self.input_dim = input_dim
        self.hidden_dim = hidden_dim
        self.batch_size = batch_size
        self.num_layers = num_layers

        #Define the initial linear hidden layer
        self.init_linear = nn.Linear(self.input_dim, self.input_dim)

        # Define the LSTM layer
        self.lstm = nn.LSTM(self.input_dim, self.hidden_dim, self.num_layers, batch_first=True, bidirectional=True)

        # Define the output layer
        self.linear = nn.Linear(self.hidden_dim * 2, output_dim)

    def init_hidden(self):
        # This is what we'll initialise our hidden state as
        return (torch.zeros(self.num_layers, self.batch_size, self.hidden_dim),
                torch.zeros(self.num_layers, self.batch_size, self.hidden_dim))

    def forward(self, input):
        #Forward pass through initial hidden layer
        linear_input = self.init_linear(input)

        # Forward pass through LSTM layer
        # shape of lstm_out: [batch_size, input_size ,hidden_dim]
        # shape of self.hidden: (a, b), where a and b both
        # have shape (batch_size, num_layers, hidden_dim).
        lstm_out, self.hidden = self.lstm(linear_input)

        # Can pass on the entirety of lstm_out to the next layer if it is a seq2seq prediction
        y_pred = self.linear(lstm_out)
        return y_pred

In [58]:
bilstm = BiLSTM(3, 16, n_classes)
optimizer = optim.SGD(bilstm.parameters(), lr=0.001, momentum=0.9)
criterion = nn.CrossEntropyLoss()

In [ ]:
for i in range(100):
    
    out = bilstm(X_tensor[:,:])
    optimizer.zero_grad()
    loss = criterion(out, y_ohe[:,:])
    loss.backward()
    print(loss)
    optimizer.step()

    # accuracy
    _, predicted = torch.max(out, 1)

In [29]:
out.shape

torch.Size([1, 43468, 11])

In [41]:
y_hat = out.argmax(dim=2)

In [42]:
y_idx = y_ohe.argmax(dim=2)

In [53]:
(y_hat == y_idx).sum()/y_hat.shape[1]

tensor(0.0136)